In [2]:
import numpy as np
import sys
import pickle
from collections import defaultdict
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data, test_classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV

import my_tools 


### define constants for base feature names
FINANCIAL_FEATURES = ["bonus", "deferral_payments", "deferred_income", "director_fees",
                      "exercised_stock_options", "expenses", "loan_advances",
                      "long_term_incentive", "other", "restricted_stock", 
                      "restricted_stock_deferred", "salary", "total_payments",
                      "total_stock_value"]

EMAIL_FEATURES = ["from_messages", "to_messages"]
EMAIL_POI_FEATURES = ["from_poi_to_this_person", "from_this_person_to_poi", "shared_receipt_with_poi"]

## These financial features have data point in at least 50% of rows
FINANCIAL_FEATURES_50PCT = ["bonus", "exercised_stock_options", "expenses",
                      "other", "restricted_stock", "salary", "total_payments",
                      "total_stock_value"]

### These are the features that appear on visual inspection to have normal-shaped distribution after
### log-transform
FINANCIAL_FEATURES_LOG_NL_SHAPE = ["bonus", "exercised_stock_options", "restricted_stock", 
                       "salary", "total payments", "total_stock_value"  ]


### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers
### 2 entries are not individuals. one labeled "TOTAL" and the other
### "THE TRAVEL AGENCY IN THE PARK"
del data_dict["TOTAL"]
del data_dict["THE TRAVEL AGENCY IN THE PARK"]

### Unclear whether other outliers should be removed



### Store to my_dataset for easy export below.
my_dataset = data_dict


In [3]:
### this is a list of all the base features supplied by Udacity
base_features_list = ['poi'] + FINANCIAL_FEATURES + EMAIL_FEATURES + EMAIL_POI_FEATURES

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, base_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

### In this step, fit a SKB selector to all of the feqtures and print out the feature names in order of
### importance.  Note: this is fitting to ALL of the data,
### which really isn't kosher, since one would only have a training set for any fitting step in a real
### application.  
skb = SelectKBest(score_func = f_classif, k=len(base_features_list)-1)
skb.fit(features, labels)
### create a new sorted feature list
skb_features_list = []
feature_scores = dict(zip(base_features_list[1:], skb.scores_))
for key in sorted(feature_scores, key=feature_scores.get, reverse = True):
    print key,":\t", feature_scores[key]
    skb_features_list.append(key)

exercised_stock_options :	24.8150797332
total_stock_value :	24.1828986786
bonus :	20.7922520472
salary :	18.2896840434
deferred_income :	11.4584765793
long_term_incentive :	9.92218601319
restricted_stock :	9.21281062198
total_payments :	8.77277773009
shared_receipt_with_poi :	8.58942073168
loan_advances :	7.18405565829
expenses :	6.09417331064
from_poi_to_this_person :	5.24344971337
other :	4.187477507
from_this_person_to_poi :	2.38261210823
director_fees :	2.12632780201
to_messages :	1.64634112944
deferral_payments :	0.224611274736
from_messages :	0.169700947622
restricted_stock_deferred :	0.0654996529099


In [5]:
### Use a pipeline and gridsearchCV to pick best number of features from base features list. F1 score as
### evaluation metric for scoring the combination.
skb = SelectKBest(score_func = f_classif)
gnb = GaussianNB()
pl = Pipeline([('skb', skb), ('gnb', gnb)])
### Use SSS cross-validator. To save time while testing, n_iter is 100, not 1000 as
### in the Udacity tester.py code.
cv = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
param_grid = {'skb__k' : [x for x in range(1, len(base_features_list))]}
gs = GridSearchCV(pl, param_grid, scoring = 'f1', cv = cv)
gs.fit(features, labels)
print "Best Estimator:", gs.best_estimator_
print "F1 score:", gs.best_score_

Best Estimator: Pipeline(steps=[('skb', SelectKBest(k=5, score_func=<function f_classif at 0x00000000072CB978>)), ('gnb', GaussianNB())])
F1 score: 0.398571428571


In [6]:
### Pass this combination to the tester.py algorithm and print results
skb = SelectKBest(k = 5, score_func = f_classif)
gnb = GaussianNB()
pl = Pipeline([('skb', skb), ('gnb', gnb)])
print "Testing pipeline SelectKBest(k=5) -> Gaussian(NB), using base_features_list features."
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(pl, my_dataset, base_features_list, folds = 1000)
print "Accuracy \t Precision \t Recall \t F1 \t\t F2"
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing pipeline SelectKBest(k=5) -> Gaussian(NB), using base_features_list features.
Accuracy 	 Precision 	 Recall 	 F1 		 F2
0.849 		 0.422 		 0.359 		 0.388 		 0.370


The accuracy and precision are already over the requirement for the project.  At this point we know what best number of features is, but not which ones actually used.  This can be different on each pass through the test algorithm, as the training features against which SelectKBest runs will be different each time.  Count how many times each feature is chosen and print sorted list of the features.

In [7]:
### Set up features
data = featureFormat(my_dataset, base_features_list, sort_keys = True )
labels, features = targetFeatureSplit(data)
### get arrays of train/test indices from StratifiedShuffleSplit
cv = StratifiedShuffleSplit(labels, n_iter = 1000, random_state = 42)
### save the feature counts in this dictionary
features_use_count = np.zeros(len(base_features_list)-1, dtype = int)
### get the indices from each iteration and create the training matrices
for train_indices, test_indices in cv:
    labels_train = []
    features_train = []
    for train_ix in train_indices:
        labels_train.append(labels[train_ix])
        features_train.append(features[train_ix])
    skb = SelectKBest(score_func = f_classif, k = 5)
    skb.fit(features_train, labels_train)
    for ix in skb.get_support(indices = True):
        features_use_count[ix] += 1

feature_scores = dict(zip(base_features_list[1:],features_use_count))
for key in sorted(feature_scores, key=feature_scores.get, reverse = True):
    print key,":\t", feature_scores[key]



total_stock_value :	1000
exercised_stock_options :	998
bonus :	997
salary :	989
deferred_income :	506
long_term_incentive :	190
restricted_stock :	133
shared_receipt_with_poi :	109
total_payments :	58
expenses :	13
from_poi_to_this_person :	7
to_messages :	0
deferral_payments :	0
loan_advances :	0
restricted_stock_deferred :	0
from_messages :	0
other :	0
from_this_person_to_poi :	0
director_fees :	0


Interesting to note that an email feature, shared_receipt_with_poi, is a little higher in this list than in the feature rankings done by SelectKBest against all of the features.

In [8]:
### Create a features list with just the top five in terms of use count, and feed that through the tester again.
test_features_list = ['poi', 'total_stock_value', 'bonus', 'exercised_stock_options', 'salary', 'deferred_income']
print "Testing GaussianNB with features:", test_features_list[1:]
clf = GaussianNB()
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(clf, my_dataset, test_features_list, folds = 1000)
print "Accuracy \t Precision \t Recall \t F1 \t\t F2"
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing GaussianNB with features: ['total_stock_value', 'bonus', 'exercised_stock_options', 'salary', 'deferred_income']
Accuracy 	 Precision 	 Recall 	 F1 		 F2
0.855 		 0.489 		 0.381 		 0.428 		 0.398


Which is slightly higher than the performance of the pipeline.

In [9]:
### Of top 5 in this list, 4 have more normal appearing shape with log transform  Apply this and try
### these features to see if any improvement.  

### Create new features for each of the financial features whose distribution has a more normal shaped
### histogram with a log transform. Substitute this for the base features and see what performance is like.

LOG_NORMAL_FEATURES = []
for feature in FINANCIAL_FEATURES_LOG_NL_SHAPE:
    my_tools.create_log_feature(my_dataset, feature, abs_value = True) 
    LOG_NORMAL_FEATURES.append("log_" + feature)

test_features_list = ['poi'] + ['log_bonus', 'deferred_income', 'log_exercised_stock_options', 
                           'log_salary', 'log_total_stock_value']
print "Testing modified feature list using log-transforms of bonus, exercised_stock_options,"
print "salary, and total_stock_value." 
print "accuracy \t precision \t recall \t f1 \t\t f2"
clf = GaussianNB()
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(clf, my_dataset, test_features_list, folds = 1000)
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing modified feature list using log-transforms of bonus, exercised_stock_options,
salary, and total_stock_value.
accuracy 	 precision 	 recall 	 f1 		 f2
0.859 		 0.511 		 0.225 		 0.312 		 0.253


In [10]:
### So that didn't seem to help.  Substitute  all the log-transformed features back into the SelectKBest
### routine and see if there is any improvement.

test_features_list = ['poi'] + ["log_bonus", "deferral_payments", "deferred_income", 
                           "director_fees", "log_exercised_stock_options", "expenses", 
                           "loan_advances", "long_term_incentive", "other", "restricted_stock", 
                           "restricted_stock_deferred", "log_salary", 
                           "total_payments", "log_total_stock_value"] + \
                            EMAIL_FEATURES + EMAIL_POI_FEATURES

### Extract features and labels from dataset again for testing
data = featureFormat(my_dataset, test_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

skb = SelectKBest(score_func = f_classif)
gnb = GaussianNB()
pl = Pipeline([('skb', skb), ('gnb', gnb)])
### Use SSS cross-validator. For purposes of testing, this is same as in the Udacity tester.py, except
### n_iter is 100 instead of 1000, for the purpose of saving time.  
cv = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
param_grid = {'skb__k' : [x for x in range(1, len(base_features_list))]}
gs = GridSearchCV(pl, param_grid, scoring = 'f1', cv = cv)
gs.fit(features, labels)
print "Best Estimator:", gs.best_estimator_
print "F1 score:", gs.best_score_

Best Estimator: Pipeline(steps=[('skb', SelectKBest(k=19, score_func=<function f_classif at 0x00000000072CB978>)), ('gnb', GaussianNB())])
F1 score: 0.278939282939


So adding specific log-transformed features, or substituting all didn't improve the performance of a Gaussian Naive Bayes classifier.  Now use the email feature which was the highest in the ranking by feature count: shared_receipt_with_poi

In [11]:
### Testing adding highest-ranked email feature to list of top 5 financial features
test_features_list = ['poi'] + ['bonus', 'deferred_income', 'exercised_stock_options', 
                           'salary', 'total_stock_value'] + ["shared_receipt_with_poi"]

print "Testing GaussianNB with features:", test_features_list[1:]
print "accuracy \t precision \t recall \t f1 \t\t f2"
clf = GaussianNB()
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(clf, my_dataset, test_features_list, folds = 1000)
print " %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)
    

Testing GaussianNB with features: ['bonus', 'deferred_income', 'exercised_stock_options', 'salary', 'total_stock_value', 'shared_receipt_with_poi']
accuracy 	 precision 	 recall 	 f1 		 f2
 0.852 		 0.474 		 0.363 		 0.411 		 0.381


That doesn't score as well as just the top 5 financial features alone.  Try adding a ratio feature which normalizes this by dividing it by "to_messages", presumably the total messages received by any one individual.

In [12]:
### create the new feature, add to dataset, run the cross-validation scorer
my_tools.create_ratio_feature(my_dataset, "shared_receipt_ratio", "shared_receipt_with_poi", "to_messages")
test_features_list = ['poi'] + ['bonus', 'deferred_income', 'exercised_stock_options', 
                           'salary', 'total_stock_value'] + ["shared_receipt_ratio"]
clf = GaussianNB()
print "Testing GaussianNB with features:", test_features_list[1:]
print "accuracy \t precision \t recall \t f1 \t\t f2"
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(clf, my_dataset, test_features_list, folds = 1000)
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing GaussianNB with features: ['bonus', 'deferred_income', 'exercised_stock_options', 'salary', 'total_stock_value', 'shared_receipt_ratio']
accuracy 	 precision 	 recall 	 f1 		 f2
0.861 		 0.516 		 0.386 		 0.441 		 0.406


Which is better than the previous best of:

Accuracy 	 Precision 	 Recall 	 F1 		 F2

0.855 		 0.489 		 0.381 		 0.428 		 0.398

In [13]:
### Try making another ratio feature: <total poi-related messages>/<all messages>
my_tools.create_ratio_feature(my_dataset, "poi_email_ratio", EMAIL_POI_FEATURES, EMAIL_FEATURES)
test_features_list = ['poi'] + ['bonus', 'deferred_income', 'exercised_stock_options', 
                           'salary', 'total_stock_value'] + ["poi_email_ratio"]
clf = GaussianNB()
print "Testing GaussianNB with feature list:", test_features_list[1:]
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(clf, my_dataset, test_features_list, folds = 1000)
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing GaussianNB with feature list: ['bonus', 'deferred_income', 'exercised_stock_options', 'salary', 'total_stock_value', 'poi_email_ratio']
0.861 		 0.516 		 0.386 		 0.441 		 0.406


So adding either email feature improves overall performance the same amount.

In [14]:
### That's the same as the first email ratio feature.
### One more approach to feature selection: do it by hand.  Select financial features which have the highest
### ratio of the median(poi)/median(non-poi) in the zero-substituted dataset.  Test increasing number of those
### features, but always include the "shared_receipt_ratio" feature.
HIGH_MEDIAN_RATIO_FEATURES = ["other", "bonus", "expenses", "restricted_stock", "total_stock_value",
                             "exercised_stock_options", "total_payments", "salary"]
gnb = GaussianNB()
i = 0
test_features_list = ['poi', 'shared_receipt_ratio']
print "n_features \t accuracy \t precision \t recall \t f1 \t\t f2"
for feature in HIGH_MEDIAN_RATIO_FEATURES:
    test_features_list = test_features_list + [feature]
    i += 1
    accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(gnb, my_dataset, test_features_list, folds = 1000)
    print " %d \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(i, accuracy, precision, recall, f1, f2)

n_features 	 accuracy 	 precision 	 recall 	 f1 		 f2
 1 		 0.807 		 0.013 		 0.002 		 0.003 		 0.002
 2 		 0.810 		 0.327 		 0.133 		 0.189 		 0.150
 3 		 0.828 		 0.341 		 0.123 		 0.181 		 0.142
 4 		 0.820 		 0.236 		 0.116 		 0.156 		 0.129
 5 		 0.856 		 0.433 		 0.246 		 0.314 		 0.269
 6 		 0.856 		 0.436 		 0.273 		 0.336 		 0.295
 7 		 0.851 		 0.398 		 0.231 		 0.292 		 0.252
 8 		 0.847 		 0.371 		 0.211 		 0.269 		 0.231


In [15]:
### So that approach doesn't achieve goal.  Now since NB assumes feature independence,
### and some of the financial features are highly correlated, run a PCA to make features as orthogonal
### as possible.  Do grid search on FINANCIAL_FEATURES first

### reload the features
test_features_list = ['poi'] + FINANCIAL_FEATURES
data = featureFormat(my_dataset, test_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

mms = MinMaxScaler()
pca = PCA()
gnb = GaussianNB()
cv = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
pl = Pipeline([('scaler', mms),('pca', pca), ('gnb', gnb) ])
param_grid = {'pca__n_components' : [x for x in range(1,len(test_features_list))]}
gs = GridSearchCV(pl, param_grid, scoring = 'f1', cv = cv)
gs.fit(features, labels)
print "Best Estimator:", gs.best_estimator_
print "F1 score for this estimator:", gs.best_score_


Best Estimator: Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, n_components=7, whiten=False)), ('gnb', GaussianNB())])
F1 score for this estimator: 0.387666666667


In [16]:
### reload the features, now use all base features
test_features_list = base_features_list
data = featureFormat(my_dataset, test_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

mms = MinMaxScaler()
pca = PCA()
gnb = GaussianNB()
cv = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
pl = Pipeline([('scaler', mms),('pca', pca), ('gnb', gnb) ])
param_grid = {'pca__n_components' : [x for x in range(1,len(test_features_list))]}
gs = GridSearchCV(pl, param_grid, scoring = 'f1', cv = cv)
gs.fit(features, labels)
print "Best Estimator:", gs.best_estimator_
print "F1 score for this estimator:", gs.best_score_


Best Estimator: Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, n_components=11, whiten=False)), ('gnb', GaussianNB())])
F1 score for this estimator: 0.396619047619


In [17]:
### reload the features, use top five financial features
test_features_list = ['poi', 'total_stock_value', 'bonus', 'exercised_stock_options', 'salary', 'deferred_income']
data = featureFormat(my_dataset, test_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

mms = MinMaxScaler()
pca = PCA()
gnb = GaussianNB()
cv = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
pl = Pipeline([('scaler', mms),('pca', pca), ('gnb', gnb) ])
param_grid = {'pca__n_components' : [x for x in range(1,len(test_features_list))]}
gs = GridSearchCV(pl, param_grid, scoring = 'f1', cv = cv)
gs.fit(features, labels)
print "Best Estimator:", gs.best_estimator_
print "F1 score for this estimator:", gs.best_score_


Best Estimator: Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, n_components=2, whiten=False)), ('gnb', GaussianNB())])
F1 score for this estimator: 0.486666666667


In [18]:
### reload the features, use top five financial features + shared_receipt_ratio
test_features_list = ['poi', 'total_stock_value', 'bonus', 'exercised_stock_options', 'salary', 'deferred_income',
                     "shared_receipt_ratio"]
data = featureFormat(my_dataset, test_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

mms = MinMaxScaler()
pca = PCA()
gnb = GaussianNB()
cv = StratifiedShuffleSplit(labels, n_iter = 100, random_state = 42)
pl = Pipeline([('scaler', mms),('pca', pca), ('gnb', gnb) ])
param_grid = {'pca__n_components' : [x for x in range(1,len(test_features_list))]}
gs = GridSearchCV(pl, param_grid, scoring = 'f1', cv = cv)
gs.fit(features, labels)
print "Best Estimator:", gs.best_estimator_
print "F1 score for this estimator:", gs.best_score_


Best Estimator: Pipeline(steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('pca', PCA(copy=True, n_components=4, whiten=False)), ('gnb', GaussianNB())])
F1 score for this estimator: 0.460333333333


In [19]:
### Run the best estimator from above through full testing
### reload the features, use top five financial features
test_features_list = ['poi', 'total_stock_value', 'bonus', 'exercised_stock_options', 'salary', 'deferred_income']
data = featureFormat(my_dataset, test_features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

mms = MinMaxScaler()
pca = PCA(n_components = 2)
gnb = GaussianNB()
pl = Pipeline([('scaler', mms),('pca', pca), ('gnb', gnb) ])
print "Testing pipeline MinMaxScaler -> PCA(n_components = 2) -> GaussianNB, with feature list:"
print test_features_list
print "accuracy \t precision \t recall \t f1 \t\t f2"
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(pl, my_dataset, test_features_list, folds = 1000)
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing pipeline MinMaxScaler -> PCA(n_components = 2) -> GaussianNB, with feature list:
['poi', 'total_stock_value', 'bonus', 'exercised_stock_options', 'salary', 'deferred_income']
accuracy 	 precision 	 recall 	 f1 		 f2
0.877 		 0.616 		 0.371 		 0.463 		 0.403


In [20]:
### Reload dataset, create custom features, run custom imputer on salary feature, test classifier
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers
### 2 entries are not individuals. one labeled "TOTAL" and the other
### "THE TRAVEL AGENCY IN THE PARK"
del data_dict["TOTAL"]
del data_dict["THE TRAVEL AGENCY IN THE PARK"]

### Unclear whether other outliers should be removed

### Store to my_dataset for easy export below.
my_dataset = data_dict

my_tools.my_imputer(my_dataset, "salary", strategy = 'median', test = my_tools.is_not_director)

my_tools.create_ratio_feature(my_dataset, "shared_receipt_ratio", "shared_receipt_with_poi", "to_messages")

test_features_list = ['poi'] + ['bonus', 'deferred_income', 'exercised_stock_options', 
                           'salary', 'total_stock_value'] + ["shared_receipt_ratio"]
clf = GaussianNB()
print "Testing GaussianNB, custom imputation, with features:", test_features_list[1:]
print "accuracy \t precision \t recall \t f1 \t\t f2"
accuracy, precision, recall, f1, f2 = my_tools.my_test_classifier(clf, my_dataset, test_features_list, folds = 1000)
print "%1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f \t\t %1.3f" %(accuracy, precision, recall, f1, f2)

Testing GaussianNB, custom imputation, with features: ['bonus', 'deferred_income', 'exercised_stock_options', 'salary', 'total_stock_value', 'shared_receipt_ratio']
accuracy 	 precision 	 recall 	 f1 		 f2
0.865 		 0.490 		 0.364 		 0.418 		 0.384


So this doesn't improve performance, although it still meets goal.